In [1]:
from dask.distributed import Client
from edag.cluster import EDAGCluster

In [2]:
cluster = EDAGCluster(username="sofroniewn", n_workers=1)
client = Client(cluster)

In [37]:
import dask.array as da
def load_data(shape):
    ndim = len(shape)
    chunks = (1, ) * (ndim - 2) + (None, None)
    return da.random.random(shape, chunks=chunks)

In [38]:
raw = client.submit(load_data, (50, 100, 64, 64))

In [39]:
raw

<Future: finished, type: dask.Array, key: load_data-6e5801df5066844af88539b7a9f2aecb>

In [40]:
client.gather(raw)

,Array,Chunk
Bytes,163.84 MB,32.77 kB
Shape,"(50, 100, 64, 64)","(1, 1, 64, 64)"
Count,5000 Tasks,5000 Chunks
Type,float64,numpy.ndarray


In [11]:
def install():
    import os
    os.system("pip install scikit-image")

In [12]:
client.run(install);  # Run on all workers

In [54]:
from skimage.filters import gaussian

def smooth(data, sigma):
    return data.map_blocks(gaussian, sigma=sigma)

In [55]:
smoothed = client.submit(smooth, raw, 1)

In [56]:
smoothed

<Future: finished, type: dask.Array, key: smooth-d723860bdf0ae8430a7dc853022cf8ac>

In [57]:
client.gather(smoothed)

,Array,Chunk
Bytes,163.84 MB,32.77 kB
Shape,"(50, 100, 64, 64)","(1, 1, 64, 64)"
Count,10000 Tasks,5000 Chunks
Type,float64,numpy.ndarray


### napari code

In [21]:
%gui qt

In [22]:
import napari

In [24]:
viewer = napari.Viewer(title='edag-dask')

In [15]:
viewer.title = 'edag-dask'

In [52]:
viewer.add_image(client.gather(raw), name='raw', contrast_limits=[0, 1])

<Image layer 'raw' at 0x147149208>

In [58]:
viewer.add_image(client.gather(smoothed), name='smoothed', contrast_limits=[0, 1])

<Image layer 'smoothed' at 0x14450a0f0>

### interactive smoothing

In [62]:
def slider_callback(viewer, value):
    sigma = value / 10
    smoothed = client.submit(smooth, raw, sigma)
    viewer.layers['smoothed'].data = client.gather(smoothed)
    viewer.status = str(sigma)

# Make a horizontal slider from 0 - 100
from qtpy.QtCore import Qt
from qtpy.QtWidgets import QSlider
slider = QSlider(Qt.Horizontal)
slider.setMinimum(0)
slider.setMaximum(100)
slider.setSingleStep(1)

# Connect your slider to callback function
slider.valueChanged.connect(lambda value: slider_callback(viewer, value))

# Add your slider as a new dockwidget
# note that you could have created a much more complex QtWidget here
# with multiple widgets inside of it etc.
viewer.window.add_dock_widget(slider, name='sigma', area='left')

In [61]:
slider.valueChanged.connect(lambda value: slider_callback(viewer, value))

### shutdown

In [63]:
client.cluster.scale(0)
client.shutdown()

distributed.batched - ERROR - Error in batched write
Traceback (most recent call last):
  File "/Users/nsofroniew/opt/anaconda3/envs/edag-dask/lib/python3.7/site-packages/distributed/batched.py", line 93, in _background_send
    payload, serializers=self.serializers, on_error="raise"
  File "/Users/nsofroniew/opt/anaconda3/envs/edag-dask/lib/python3.7/site-packages/tornado/gen.py", line 735, in run
    value = future.result()
  File "/Users/nsofroniew/opt/anaconda3/envs/edag-dask/lib/python3.7/site-packages/edag/dask_ws/comm.py", line 97, in write
    await stream.send(struct.pack("Q", len(frames)))
  File "/Users/nsofroniew/opt/anaconda3/envs/edag-dask/lib/python3.7/site-packages/websockets/protocol.py", line 555, in send
    await self.ensure_open()
  File "/Users/nsofroniew/opt/anaconda3/envs/edag-dask/lib/python3.7/site-packages/websockets/protocol.py", line 803, in ensure_open
    raise self.connection_closed_exc()
websockets.exceptions.ConnectionClosedError: code = 1011 (unexpect

In [65]:
111

111